In [39]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import sklearn
import json
import ast
import math

In [2]:
df = pd.read_csv("../data/play_by_play_fbs_regular_all.csv")

In [3]:
np.random.seed(12)

In [4]:
sample = df.sample(100_000).copy()

In [8]:
def time_remaining(row):
    period = int(row.period)
    time = ast.literal_eval(row.clock)
    remaining = ((4 - period) * 15 * 60) + (time['minutes'] * 60) + time['seconds']
    return remaining

sample['time_remaining'] = sample[['period', 'clock']].apply(time_remaining, axis=1)

In [16]:
def score(row):
    off_score = int(row.offense_score)
    def_score = int(row.defense_score)
    return off_score - def_score
    
sample['score'] = sample[['offense_score', 'defense_score']].apply(score, axis=1)

In [62]:
# Adjusted Score = Score / sqrt(Seconds + 1)
def adjusted_score(row):
    time_remaining = row.time_remaining + 1
    score = row.score
    if time_remaining < 0:
        adj_score = None
    else:
        adj_score = score / math.sqrt(time_remaining)
    return adj_score
    
sample['adj_score'] = sample[['score', 'time_remaining']].apply(adjusted_score, axis=1)